# Azure Cognitive Services Language Services

In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

client = TextAnalyticsClient('ANALTYICS_ENDPOINT', AzureKeyCredential('ANALYTICS_KEY'))

In [2]:
input = [{ 'id': '1000', 'text': 'Programming is fun, but the hours are long' }]
response = client.analyze_sentiment(input)

for result in response:
    print(result.confidence_scores)

{'positive': 0.85, 'neutral': 0.03, 'negative': 0.12}


In [3]:
from azure.core.exceptions import AzureError

try:
    input = [{ 'id': '1000', 'text': 'Programming is fun, but the hours are long' }]
    response = client.analyze_sentiment(input)

    for result in response:
        print(result.confidence_scores)
        
except AzureError as e:
    print(e.message)

{'positive': 0.85, 'neutral': 0.03, 'negative': 0.12}


In [4]:
input = [
    { 'id': '1000', 'text': 'Programming is fun, but the hours are long' },
    { 'id': '1001', 'text': 'Great food and excellent service' },
    { 'id': '1002', 'text': 'The product worked as advertised but is overpriced' },
    { 'id': '1003', 'text': 'Moving to the cloud was the best decision we ever made' },
    { 'id': '1004', 'text': 'Programming is so fun I\'d do it for free. Don\'t tell my boss!' }
]

response = client.analyze_sentiment(input)

for result in response:
    text = ''.join([x.text for x in result.sentences])
    print(f'{text} => {result.confidence_scores.positive}')

Programming is fun, but the hours are long => 0.85
Great food and excellent service => 1.0
The product worked as advertised but is overpriced => 0.01
Moving to the cloud was the best decision we ever made => 0.99
Programming is so fun I'd do it for free. Don't tell my boss! => 0.99


In [5]:
documents = [
    'My printer isn\'t working. Can someone from IT come to my office \
    and have a look?'
]

results = client.recognize_entities(documents)

for result in results:
    for entity in result.entities:
        print(f'{entity.text} ({entity.category})')

printer (Product)
IT (Skill)
office (Location)


In [6]:
documents = [
    'Natural Language Processing, or NLP, encompasses a variety of activities \
    including text classification, keyword extraction, named-entity recognition, \
    question answering, and language translation.'
]

results = client.extract_key_phrases(documents)

for result in results:
    for phrase in result.key_phrases:
        print(phrase)

Natural Language Processing
language translation
text classification
keyword extraction
question answering
NLP
variety
activities
recognition


In [7]:
import requests

headers = {
    'Ocp-Apim-Subscription-Key': 'TRANSLATION_KEY',
    'Ocp-Apim-Subscription-Region': 'TRANSLATION_REGION',
    'Content-type': 'application/json'
}

input = [{ 'text': 'Quand votre nouveau livre sera-t-il disponible?' }]
uri = 'TRANSLATION_ENDPOINT' + 'detect?api-version=3.0&to=en'
response = requests.post(uri, headers=headers, json=input)
results = response.json()

print(results[0]['language'])

fr


In [8]:
input = [{ 'text': 'Quand votre nouveau livre sera-t-il disponible?' }]
uri = 'TRANSLATION_ENDPOINT' + 'translate?api-version=3.0&to=en'
response = requests.post(uri, headers=headers, json=input)
results = response.json()

print(results[0]['translations'][0]['text'])

When will your new book be available?
